In [24]:
import re
import pandas as pd
pd.set_option('display.max_rows', None)
with open('output-grid-causal-2.txt') as fhand:
    file = fhand.read()
op = re.split(r'\n\n\n',file)
op[0]

'parameter(adv_step, adv_epoch, adv_epoch_full, lr, ro): (2048.0, 10.0, 10.0, 1e-06, 1e-06)\ngender/race combined consistency 0\nspouse consistency 0\nacc_temp: 0.39115533443891654, bal_acc_temp: 0.4883667542293745, race_gap_rms_temp: 0.007396053779178127, race_max_gap_temp: 0.008405974510597547, gender_gap_rms_temp: 0.18381555276619255, gender_max_gap_temp: 0.2108603170665478'

In [25]:
def string_format(x):
    s = re.findall(r'(\d+)\.(\d+)|(\d+)e-(\d+)|(\d)\.(\d+)e-(\d+)|(\d+)',x)
    string_s = []
    for i in s:
        if i[0] != '':
            string_s.append(i[0]+'.'+i[1])
        if i[2] != '':
            string_s.append(i[2] + 'e-'+i[3])
        if i[4] != '':
            string_s.append(i[4]+'.'+i[5]+'e-'+i[6])
        if i[7] !='':
            string_s.append(i[7])
    return(string_s)
ops = [string_format(x) for x in op]
ops = ops[:(len(ops)-1)]

In [26]:
output_dict = dict()
for op in ops:
    for i in range(len(op)):
        op[i] = float(op[i])
    output_dict[tuple(op[:5])] = op[5:]

In [27]:
df_output = pd.DataFrame.from_dict(output_dict, orient='index', columns=['gender/race combined consistency','spouse consistency', 'acc-temp' , 'bal-acc-temp', 'race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.rename_axis('(adv_step, adv_epoch, adv_epoch_full, lr, ro)')
df_output = df_output.sort_values(by=['race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.sort_values(by=['bal-acc-temp'],ascending=False)

Outputs for adult data with sensr algorithm. Sensetive directions (directions for both gender and race) are from causal regression: X_gender_race_train ~ y_gender_race_train

In [28]:
df_out_selected = df_output[df_output['bal-acc-temp']>0.77]
df_out_selected = df_out_selected[df_out_selected['gender-max-gap']<0.17]
df_out_selected[['acc-temp' , 'bal-acc-temp', 'gender-gap-rms', 'race-gap-rms',  'gender-max-gap', 'race-max-gap']]

,acc-temp,bal-acc-temp,gender-gap-rms,race-gap-rms,gender-max-gap,race-max-gap
"(adv_step, adv_epoch, adv_epoch_full, lr, ro)",,,,,,
"(4.0, 10.0, 50.0, 0.008, 0.0001)",0.804754,0.821631,0.121325,0.046685,0.164891,0.065546
"(4.0, 10.0, 50.0, 0.01, 0.0001)",0.804754,0.821322,0.121806,0.047028,0.169331,0.064089
"(8.0, 10.0, 20.0, 0.002, 0.0001)",0.805860,0.820972,0.114575,0.029726,0.157277,0.040021
"(4.0, 10.0, 60.0, 0.01, 0.0001)",0.810724,0.820481,0.116419,0.041471,0.159464,0.057169
"(8.0, 10.0, 80.0, 0.004, 0.0001)",0.803317,0.820064,0.122556,0.036681,0.159666,0.051536
"(8.0, 10.0, 60.0, 0.002, 0.0001)",0.807076,0.819151,0.124732,0.034499,0.164722,0.048521
"(4.0, 10.0, 80.0, 0.01, 0.0001)",0.800663,0.818774,0.122770,0.048413,0.163661,0.068438
"(8.0, 10.0, 60.0, 0.004, 0.0001)",0.798784,0.818150,0.119294,0.052184,0.164377,0.070959
"(8.0, 10.0, 40.0, 0.004, 0.0001)",0.802543,0.817083,0.110841,0.048980,0.153213,0.062211
